In [1]:
import json
import os
import pandas as pd

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open(os.path.join("./results", "data_dict.json"), "r") as f:
    res_obj = json.load(f)

In [3]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

dir_name = "far_shah_b1-b5_b8_train_EOI_split"

with open(os.path.join("./", dir_name, "test/result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [4]:
res_obj[0]

{'file_name': '/home/dtron2_user/ls_dtron2_full/model/far_shah_b1-b5_b8_train_EOI_split/test/images/B7XiQazXSAuk91Fp-lawschooltransparency_ss.png',
 'height': 1080,
 'width': 1920,
 'annotations': [{'iscrowd': 0,
   'bbox': [919.9862, 26.2133, 1001.4822, 53.5079],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [593.9142, 27.0521, 726.9221, 51.7033],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [778.6895, 25.5116, 866.3443, 54.6226],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [315.9889, 536.3185, 570.6516, 594.7294],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1477.5504, 929.0659, 1755.1201, 982.048],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1547.1481, 26.9978, 1611.3625, 54.8113],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [1480.5657, 856.9562, 1755.4635, 925.6236],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [

In [5]:
coco_gt_obj["annotations"][0]

{'id': 112366,
 'image_id': 8514,
 'category_id': 1,
 'segmentation': [],
 'bbox': [455.6712328767122,
  51.78082191780822,
  312.16438356164383,
  50.3013698630137],
 'ignore': 0,
 'iscrowd': 0,
 'area': 15702.29611559392}

In [6]:
# Group By Image

imgs_gt_pred_list = {}

tot_num_pred = 0
tot_num_gt = 0

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"][2:]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, x1 + w, y1 + h])
            tot_num_gt += 1
    
    pred_list = []
    
    for res in res_obj:
        fn2 = os.path.basename(res["file_name"])
        if fn1 == fn2:
            for ann2 in res["annotations"]:
                pred_list.append(ann2["bbox"])
                tot_num_pred += 1
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [7]:
print(tot_num_pred)
print(tot_num_gt)

46382
27515


In [8]:
keys = list(imgs_gt_pred_list.keys())


In [9]:
def calculate_iou(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [10]:
def calculate_int(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    return interArea

In [11]:
thold = .5

In [12]:
iou_scores = []

for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    for i, pred in enumerate(preds):
        for j, gt in enumerate(gts):
            iou = calculate_iou(pred, gt)
            iou_scores.append([key, i, j, iou])
                
iou_scores_df = pd.DataFrame(iou_scores, columns=["fn", "pred_ind", "gt_ind", "iou"])

In [13]:
iou_scores_df

,fn,pred_ind,gt_ind,iou
0,qicFlNeWbe16QeQV-letterland_ss.png,0,0,0.000000
1,qicFlNeWbe16QeQV-letterland_ss.png,0,1,0.742547
2,qicFlNeWbe16QeQV-letterland_ss.png,0,2,0.000000
3,qicFlNeWbe16QeQV-letterland_ss.png,0,3,0.000000
4,qicFlNeWbe16QeQV-letterland_ss.png,0,4,0.000000
...,...,...,...,...
702126,w1kHJhz3oG0oy6yu-acerill_ss.png,2,1,0.000000
702127,w1kHJhz3oG0oy6yu-acerill_ss.png,2,2,0.531452
702128,w1kHJhz3oG0oy6yu-acerill_ss.png,3,0,0.000000
702129,w1kHJhz3oG0oy6yu-acerill_ss.png,3,1,0.000000


In [14]:
iou_scores_df_g = iou_scores_df.groupby('fn')

In [15]:
iou_full_g_dfs = []
iou_corr_g_dfs = []
iou_dang_g_pred_dfs = []
iou_dang_g_gt_dfs = []
conf_mat = []

img_ctr = 0
pred_gtr_gt_list = []
gt_gtr_pred_list = []

for name, group in iou_scores_df_g:
    
    print(name)
    
    df_subset = group[["fn", "pred_ind", "gt_ind", "iou"]]
    df_subset = df_subset.sort_values(by="iou", ascending=False)
    df_subset = df_subset.reset_index(drop=True)
    
    iou_full_g_dfs.append(df_subset)
    tp = 0
    
    gt_list = list(df_subset["gt_ind"])
    pred_list = list(df_subset["pred_ind"])
        
    gt_set = list(set(gt_list))
    pred_set = list(set(pred_list))
    
    num_gt = len(gt_set)
    num_pred = len(pred_set)
    
    data_corr = []
    data_pred_dang = []
    data_gt_dang = []
    
    rem_flag = 0
    working_set = None
    
    # Number of predicted >= number of ground truths 
    if num_pred >= num_gt:
        pred_gtr_gt_list.append(img_ctr)
        working_set = pred_set
        rem_flag = 1
        
    # Number of ground truths > number of predicted    
    elif num_gt > num_pred:
        gt_gtr_pred_list.append(img_ctr)
        working_set = gt_set
        rem_flag = 2

    for i, row in df_subset.iterrows():
        
        if rem_flag == 1:
            if row["pred_ind"] in pred_set and row["iou"] > thold:
                data_corr.append(row)
                pred_set.remove(row["pred_ind"])
                tp += 1
            elif row["pred_ind"] in pred_set:
                data_pred_dang.append(row)
                pred_set.remove(row["pred_ind"])
            elif not pred_set:
                break
        elif rem_flag == 2:
            if row["gt_ind"] in gt_set and row["iou"] > thold:
                data_corr.append(row)
                gt_set.remove(row["gt_ind"])
                tp += 1
            elif row["gt_ind"] in gt_set:
                data_gt_dang.append(row)
                gt_set.remove(row["gt_ind"])
            elif not gt_set:
                break

    iou_corr_g_dfs.append(pd.DataFrame(data_corr))
    iou_dang_g_pred_dfs.append(pd.DataFrame(data_pred_dang))
    iou_dang_g_gt_dfs.append(pd.DataFrame(data_gt_dang))    
    
    fp = num_pred - tp
    fn = num_gt - tp
    
    conf_mat.append([tp, fp, fn])
    
    img_ctr += 1

0002HSj7V4ZbNIO4-publimaison_ss.png
01fbQWfpmLcG7v0v-uptiseo_ss.png
03L7CyeohRlupl82-lh-st_ss.png
03qbD23eAIS1XVPG-mygulfstream_ss.png
048KxK73nIwwLoD0-mbbfstyle_ss.png
05rfcCBbt5EBAK4D-fultoncountyoh_ss.png
06aaCGuceDo5T5rC-prototypehubs_ss.png
0DxuLq4Wsdywaj0B-readdemonslayer_ss.png
0FX4MgBuXfPglIaX-bakerbotts_ss.png
0HeWTZT3whEnG7Jq-generalpacket_ss.png
0IVx3rmYK8R8XARL-wildplay_ss.png
0OB1J4nigM9MWAwT-teracoin_ss.png
0TTV3BZ7KVkHcCET-quip_ss.png
0Vuu4DtB7ssQaonF-otp_ss.png
0Wl0D2ZttZ5u2aVS-northshorecollege_ss.png
0Wq8lNumfTA60AlK-sydneypadua_ss.png
0Z8fY051Jg7hYnPd-globaldreamers_ss.png
0c0sLBSOPVIFAU3D-trendoveragency_ss.png
0dBjOH00xLej6fhw-cases_ss.png
0eU82dovyR0tcCg6-supind_ss.png
0eWNkcHSVk9SoaKG-villageinn_ss.png
0fOtTqW4ZNUX0lmU-domainecarneros_ss.png
0iuYnYW3EZO0NswG-bankbranchlocator_ss.png
0jPnnkvk7WFEDTwq-myproviderlink_ss.png
0nZeZy1afN0hSWRN-purplewave_ss.png
0sQx11XqhFJIvXMF-iavm_ss.png
0uIdKB6z4i351YjD-habitat_ss.png
0uiCqqYhi7H8zmpw-hercmagnus_ss.png
0w6wjFWfSZQyH

In [16]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    # Would be equivalent to correctly identifying areas where no object is present... so we set it to 0 because it is not relevant
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

In [17]:
print("tp_tot", tp_tot)
print("fp_tot", fp_tot)
print("tn_tot", tn_tot)
print("fn_tot", fn_tot)

tp_tot 22054
fp_tot 24328
tn_tot 0
fn_tot 5461


In [18]:
print(len(pred_gtr_gt_list))

2016


In [19]:
print(pred_gtr_gt_list)
print(gt_gtr_pred_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 22

In [20]:
voi = 11

In [21]:
iou_full_g_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,0OB1J4nigM9MWAwT-teracoin_ss.png,2,11,0.934892
1,0OB1J4nigM9MWAwT-teracoin_ss.png,21,6,0.908107
2,0OB1J4nigM9MWAwT-teracoin_ss.png,12,9,0.862309
3,0OB1J4nigM9MWAwT-teracoin_ss.png,7,3,0.841392
4,0OB1J4nigM9MWAwT-teracoin_ss.png,17,4,0.808407
...,...,...,...,...
331,0OB1J4nigM9MWAwT-teracoin_ss.png,8,0,0.000000
332,0OB1J4nigM9MWAwT-teracoin_ss.png,7,13,0.000000
333,0OB1J4nigM9MWAwT-teracoin_ss.png,7,12,0.000000
334,0OB1J4nigM9MWAwT-teracoin_ss.png,7,11,0.000000


In [22]:
iou_corr_g_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,0OB1J4nigM9MWAwT-teracoin_ss.png,2,11,0.934892
1,0OB1J4nigM9MWAwT-teracoin_ss.png,21,6,0.908107
2,0OB1J4nigM9MWAwT-teracoin_ss.png,12,9,0.862309
3,0OB1J4nigM9MWAwT-teracoin_ss.png,7,3,0.841392
4,0OB1J4nigM9MWAwT-teracoin_ss.png,17,4,0.808407
5,0OB1J4nigM9MWAwT-teracoin_ss.png,8,12,0.748595
6,0OB1J4nigM9MWAwT-teracoin_ss.png,14,7,0.542708


In [23]:
iou_dang_g_pred_dfs[voi]

,fn,pred_ind,gt_ind,iou
7,0OB1J4nigM9MWAwT-teracoin_ss.png,13,0,0.418639
8,0OB1J4nigM9MWAwT-teracoin_ss.png,11,1,0.201941
9,0OB1J4nigM9MWAwT-teracoin_ss.png,6,10,0.185029
10,0OB1J4nigM9MWAwT-teracoin_ss.png,0,0,0.124893
11,0OB1J4nigM9MWAwT-teracoin_ss.png,18,13,0.062344
13,0OB1J4nigM9MWAwT-teracoin_ss.png,19,2,0.046608
15,0OB1J4nigM9MWAwT-teracoin_ss.png,10,13,0.000301
16,0OB1J4nigM9MWAwT-teracoin_ss.png,15,12,0.000000
22,0OB1J4nigM9MWAwT-teracoin_ss.png,16,4,0.000000
90,0OB1J4nigM9MWAwT-teracoin_ss.png,22,7,0.000000


In [24]:
iou_dang_g_gt_dfs[voi]

""


In [25]:
accuracy = (tp_tot + tn_tot) / (tp_tot + fp_tot + tn_tot + fn_tot)
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print("accuracy: ", accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1_score: ", f1_score)

accuracy:  0.4253997646741122
precision:  0.4754861799836143
recall:  0.801526440123569
f1_score:  0.5968848532416742


In [26]:
with open('./results/metrics.txt', 'w+') as file:
    file.write(f"accuracy: {accuracy}\n")
    file.write(f"precision: {precision}\n")
    file.write(f"recall: {recall}\n")
    file.write(f"f1_score: {f1_score}\n")
